## Part 2
#### Load training file and create dictionary containing the occurrences of each annotation in the training set

In [1]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [2]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 47860, 'I-positive': 1426, 'B-positive': 1135, 'I-negative': 96, 'B-negative': 111, 'I-neutral': 1369, 'B-neutral': 1350}


#### (a) Write a function that estimates the emission parameters from the training set using MLE

In [3]:
def estimate_emission(word, tag):
    countTop = 0
    countBottom = annotationDict[tag]
    for i in range(len(train_list)):
        annotated_word = train_list[i]
        each_word = annotated_word.split(" ")
        if len(each_word) == 2:
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1

#         print(countTop)
#         print(countBottom)
    return countTop/countBottom

estimate_emission("service", "O")

0.0

#### (b) Replace the words that appear less than k times in the training set with a special token #UNK# before training. This leads to a “modified training set”.

In [4]:
import datetime

def modify_trainingset(k):
        wordCountDict = {}
        for i in range(len(train_list)):
            annotated_word = train_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordCountDict:
                wordCountDict[each_word[0]] += 1
            else:
                wordCountDict[each_word[0]] = 1
        
       
        wordToBeReplacedDict = {}
        wordToBeReplacedList = []
        for key in wordCountDict:
            if wordCountDict[key] < k:
                wordToBeReplacedList.append(key)

        start = datetime.datetime.now()
        for i in range(len(train_list)): 
            annotated_word = train_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordToBeReplacedList:
                train_list[i] = "#UNK# " + each_word[1]


        modifiedTotalTrainSet = "\n".join(train_list)

        f = open("modified_train","w+")
        f.write(modifiedTotalTrainSet)
                            
        end = datetime.datetime.now()
        elapsed = end - start
        print(elapsed)

modify_trainingset(3)

0:00:07.980760


#### Loads the modified training set into a list, and count the occurences of each word in the list and store it into a dictionary called modifiedWordDict

In [5]:
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
modified_train_list = load_modified_train_file()

In [6]:
def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    
modifiedWordDict = get_word_dict()

In [7]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

    
wordTagPairDict = count_word_tag_pair_dict()

#### Implement a fixed emission function

In [8]:
def estimate_emission_fix(word, tag):
        countTop = 0
        countBottom = annotationDict[tag]
        if word in modifiedWordDict:
            # do nothing
            word = word
            
        else:
            word = "#UNK#"

        for i in range(len(modified_train_list)):
            annotated_word = modified_train_list[i]
            each_word = annotated_word.split(" ")
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1
        return countTop/countBottom

estimate_emission_fix("I", "O")

6.268282490597577e-05

#### Loop through the words once and store the emission value of each word tag pair into a giant dictionary called giantEmissionDict

In [9]:
def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict
    
giantEmissionDict = store_estimate_emission_fix()
print(giantEmissionDict)

{'我 O': 0.013455913079816131, '现在 O': 0.0008357709987463436, '想 O': 0.001420810697868784, '吃 O': 0.0007939824488090263, '蛋糕 B-neutral': 0.0007407407407407407, '， O': 0.06140827413288759, '#UNK# B-neutral': 0.44666666666666666, '菜 B-neutral': 0.0007407407407407407, '茶 I-neutral': 0.002191380569758948, '好多 O': 0.00012536564981195154, '#UNK# O': 0.18510238194734643, '啦 O': 0.0014417049728374426, '～ O': 0.0007939824488090263, '[ O': 0.01759297952361053, '污 O': 0.00012536564981195154, '] O': 0.017801922273297115, '# B-positive': 0.14537444933920704, '偶滴 I-positive': 0.014025245441795231, '歌神 I-positive': 0.023842917251051893, '啊 I-positive': 0.021037868162692847, '# O': 0.01700793982448809, '貌似 O': 0.0002089427496865859, '挺不错 O': 0.0003134141245298788, '的 O': 0.04335562055996657, '这个 O': 0.0018386961972419557, '男性 O': 0.00014625992478061012, '真的 O': 0.0016297534475553698, '哦 O': 0.000605933974091099, '了 O': 0.0150856665273715, '#UNK# I-neutral': 0.337472607742878, '？ O': 0.00254910154617634

#### (c) Implement a simple sentiment analysis system that produces the tag

In [10]:
def sentiment_analysis(word):
    all_tags = ["O", "B-positive", "I-positive", "B-negative", "I-negative", "B-neutral", "I-neutral"]
    scoreDict = {}
    # if word never appeared before, let it be #UNK#
    if word in modifiedWordDict:
        word = word
    else:
        word = "#UNK#"
    for tag in all_tags:
        possible_word_tag = word + " " + tag
        if giantEmissionDict.get(possible_word_tag) == None:
            scoreDict[tag] = 0
        else:
            scoreDict[tag] = giantEmissionDict[possible_word_tag]
            
    maximumScore = max(scoreDict.values())
    return max(scoreDict, key=scoreDict.get)

   
sentiment_analysis("ambience")

'B-negative'

#### Loads dev.in file and store as dev_in_list

In [11]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
        
dev_in_list = load_files()

In [12]:
import datetime
def predict_sentiment():
    wordDict = {}
    f = open("dev.p2.out","w+")
    giantStringToBeWritten = ""
    start = datetime.datetime.now()
    for i in range(len(dev_in_list)):
        each_word = dev_in_list[i]
        if each_word in wordDict:
            tag = wordDict[each_word]
            giantStringToBeWritten += each_word +  " " + tag + "\n"

        else:
            if each_word != "":
                tag = sentiment_analysis(each_word)
                wordDict[each_word] = tag 
                giantStringToBeWritten += each_word +  " " + tag + "\n"

            else:
                giantStringToBeWritten += " \n"


    f.write(giantStringToBeWritten)
    end = datetime.datetime.now()
    elapsed = end - start
    print(elapsed)

predict_sentiment()

0:00:00.034470


### Part 3 
#### Function to estimate Transition Parameters using MLE

#### Creating a giant_each_tweet_tag that contains only the states of each sentence

In [13]:
def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
print(totalStateNumber)

{'START': 1429, 'O': 47833, 'B-neutral': 1350, 'I-neutral': 1369, 'STOP': 1429, 'B-positive': 1134, 'I-positive': 1421, 'B-negative': 111, 'I-negative': 96}


In [14]:
def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict
    
giantTransitionDict = store_estimate_transition()
print(giantTransitionDict)

{'START O': 0.7074877536738978, 'O O': 0.927497752597579, 'O B-neutral': 0.022724897037610017, 'B-neutral O': 0.46814814814814815, 'B-neutral I-neutral': 0.5059259259259259, 'I-neutral O': 0.46092037983929873, 'O STOP': 0.029059435954257522, 'START B-positive': 0.14415675297410777, 'B-positive I-positive': 0.6031746031746031, 'I-positive I-positive': 0.5186488388458832, 'I-positive O': 0.458128078817734, 'START B-neutral': 0.13995801259622112, 'B-positive O': 0.38271604938271603, 'O B-positive': 0.01869002571446491, 'I-positive B-positive': 0.013370865587614356, 'O B-negative': 0.0020278886960884744, 'B-negative I-negative': 0.43243243243243246, 'I-negative O': 0.4791666666666667, 'I-negative I-negative': 0.5, 'B-positive B-neutral': 0.001763668430335097, 'I-neutral I-neutral': 0.5010956902848794, 'I-neutral B-positive': 0.004382761139517896, 'B-neutral B-neutral': 0.017777777777777778, 'B-negative O': 0.5675675675675675, 'B-positive STOP': 0.004409171075837742, 'I-positive B-neutral':

In [15]:
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences
sentences = split_into_sentences(dev_in_list)

## Viterbi

In [16]:
def find_path_backward(piList, sentence):
    path = []
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    if len(piList) == len(sentence):
#         print('sentence length and pilist length are same, move on')
        for i in range(len(piList)):
            # select the max of the last layer, use the max to find the previous value
            if i == 0: #its the start, i = 0 comes here
                lastLayer = piList[len(piList) - (i+1)]
                maxPiTag = max(lastLayer, key=lastLayer.get) 
                path.append('end')
            elif i == 1:
                path.append(maxPiTag)
            else: #i>0 comes here
                currentLayer = piList[len(piList) - (i+1)] #2nd last layer onwards
                targetedTag = path[i-1]
                scoreDict = {}
                for tag in currentLayer: 
                    if giantTransitionDict.get(tag + " " + targetedTag) != None:
                        scoreOfTag = giantTransitionDict[tag + " " + targetedTag] * currentLayer[tag]
                        scoreDict[tag] = scoreOfTag
                    
                bestScoreTag = max(scoreDict, key=scoreDict.get)
                path.append(bestScoreTag)
    singleStringToBeWritten = ""
    for j in range(len(sentence)):
        if sentence[j] == "":
            singleStringToBeWritten += "\n"
        else:
            singleStringToBeWritten += sentence[j] + " " + path[len(path)-(j+1)] + "\n"
    return singleStringToBeWritten

# expected: O, B-pos, I-pos, I-pos, I-pos, O, O, O
piList4 = [{'O': 0.10952242946302841, 'B-negative': 0.0019577571933592874, 'B-positive': 0.010624504726325918, 'B-neutral': 0.0009040105193951349}, {'O': 3.108819999877843e-05}, {'B-negative': 1.212411803650987e-09, 'B-positive': 3.5875587995634343e-09}, {'B-positive': 4.751733509355542e-11}]
sentence2 = ['AVOID', 'THE', 'PLACE', '']
find_path_backward(piList4, sentence2)

'AVOID O\nTHE O\nPLACE B-positive\n\n'

In [17]:
def viterbi(sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    sentence.append("") #so we can detect the end
    beginning = True
    piList = []
#     for word in sentence:
    for i in range(len(sentence)):
        word = sentence[i]
        if word in modifiedWordDict:
            word = word
        else:
            word = "#UNK#"
        
        piLayer = {}
        if beginning: #base case
            for tag in all_tags:
                if giantTransitionDict.get("START" + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                    pi = giantTransitionDict["START" + " " + tag]*giantEmissionDict[word + " " + tag]
                    piLayer[tag] = pi
            piList.append(piLayer)
            beginning = False
        else: #recursive
            previousLayer = piList[len(piList) - 1] #last entry of piList
            if word == "": #end of sentence
                tempLayer = {} 
                for previous_tag in previousLayer:
                    if giantTransitionDict.get(previous_tag + " " + "STOP") != None:
                        pi = previousLayer[previous_tag]*giantTransitionDict[previous_tag + " " + "STOP"] #new score
                        tempLayer[previous_tag] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get)
                    maximumPiScore = max(tempLayer.values())
                    lastLayer = {}
                    lastLayer[maxPiTag] = maximumPiScore
                    piList.append(lastLayer)
                    singleStringToBeWritten = find_path_backward(piList, sentence)
                    return singleStringToBeWritten
                else:
                    return "ERROR"

            else: #if not end of sentence
                for tag in all_tags:
                    tempLayer = {}
                    for previous_tag in previousLayer:
                        if giantTransitionDict.get(previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                            pi = previousLayer[previous_tag]*giantTransitionDict[previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                            tempLayer[previous_tag] = pi
                    # what if the length is 0? it will never transit to the tag
                    if len(tempLayer) != 0:
                        # pick the max here
                        maxPiTag = max(tempLayer, key=tempLayer.get) 
                        maxPiScore = max(tempLayer.values())
                        piLayer[tag] = maxPiScore
                piList.append(piLayer)


giantStringToBeWritten = ""
for sentence in sentences:
# singleStringToBeWritten = viterbi(['AVOID', 'THE', 'PLACE', ''])
    singleStringToBeWritten = viterbi(sentence)
    giantStringToBeWritten += singleStringToBeWritten
print(giantStringToBeWritten)
f = open("dev.p3.out","w+")
f.write(giantStringToBeWritten)


[ O
哈哈 O
] O
[ O
哈哈 O
] O
[ O
哈哈 O
] O
收获 O
颇丰 O
还有 O
明年 O
的 O
希腊 O
游 O
开心 O
的 O
不得了 O
http://t.cn/RyhHSQG O

# O
彭于 O
晏 O
# O
# O
方新武 O
# O
# B-positive
湄公河 I-positive
行动 I-positive
# O
@ O
彭于 O
晏 O
@ O
彭于 O
晏 O
又 O
被 O
秒 O
到 O
[ O
摊手 O
] O
[ O
摊手 O
] O
[ O
摊手 O
] O
[ O
摊手 O
] O

# B-positive
北海道 I-positive
民宿 I-positive
- I-positive
赫 O
舍里 O
## O
中华 O
料理 O
# O
来到 O
日本 O
要 O
不要 O
来 O
品尝 O
一下 O
日式 O
中国 O
菜 O
呢 O
？ O
很多 O
人 O
都 O
会 O
说 O
、 O
日本 O
的 O
美食 O
都 O
吃不完 O
呢 O
哪里 O
有 O
时间 O
吃 O
这个 O
？ O
[ O
哈哈 O
] O
[ O
哈哈 O
] O
博主 O
可以 O
理解 O
～ O
但是 O
也 O
一定 O
请 O
您 O
再次 O
光临 O
日本 O
的 O
时候 O
品尝 O
一下 O
。 O
[ O
爱 O
你 O
] O
坐在 O
异国他乡 O
的 O
中国 O
餐馆 O
品尝 O
着 O
中国 O
料理 O
必定 O
能 O
有 O
另一番 O
感觉 O
的 O

企鹅 B-neutral
杯 I-neutral
～ O
～ O
蓝绿色 O
日本 B-positive
限量 I-positive
终于 O
到手 O
[ O
好 O
喜欢 O
] O

无法 O
改变 O
的 O
计划 O
， O
有 O
[ O
yz O
招财 O
[ O
害羞 O
] O
猫 O
] O
如 O
走进 O
了 O
死胡同 O
的 O
马车 O
。 O
这种 O
计划 O
， O
令人 O
不敢恭维 O
。 O

# B-positive
易 I-positive
烊 I-positive
千玺 I-positive
汤晶媚 O
摸头 O
杀 O
# B-positive
易 I-

39010

## Viterbi try 3